### 任务

设计网络结构，反转一个变长序列（最大长度N=20），即246910000反转为196420000，其中1-9为需要反转的有效字符，0为补位字符。

#### Constraints：
1. 结构设计，不用调参，固定batch=32， lr=0.02, optimizer=Adam, epoch=1
2. 将序列中的数字映射到8维空间作为输入
3. 禁止直接将input与output层相连

#### 考核点：

#### 1. 序列长度很长时，如何记住前序信息

在序列网络中引入优化机制 - 注意力机制,如果在序列很长的情况下,不引入这个机制,识别效果是不会太好的


#### 2. output structure 是序列的优化方法

### 导入相关库

In [64]:
import os
import math
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torchtext.data import Field, BucketIterator, TabularDataset
from torch.utils.data import TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [125]:
pred  = torch.tensor([[11, 10, 11,  6,  6, 11, 12,  6, 12,  4, 10,  6, 11,  4,  5,  4,  7,  2,
          0,  0,  0],
        [ 6,  6,  7, 11,  5, 12, 10,  6, 11, 11, 11,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  6,  6, 12, 10,  4,  9,  8,  8, 10,  9, 12,  9,  4, 12,  6,  6,  6,
          2,  0,  0],
        [ 8, 10, 12,  4, 12,  9,  4, 11,  9,  6,  4,  6,  7, 12,  2,  0,  0,  0,
          0,  0,  0],
        [ 4,  6,  9,  6, 12,  8, 11,  9,  7, 10,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9, 10, 11,  7,  8,  5,  7,  5,  8,  4, 12, 12,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  8,  4,  8,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5, 11,  4, 12,  8, 12,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6,  5,  5, 11,  4, 10,  4,  6,  8,  7,  6,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  4,  4,  5,  8,  6, 12,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  6,  6,  7,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  8,  9,  7,  5,  5,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 8,  4,  9, 10,  7,  4, 11,  8, 11, 10, 12,  4,  5,  8,  5,  7, 10,  2,
          0,  0,  0],
        [ 7, 10,  8,  9,  5,  9,  4, 10, 11, 10,  8,  6,  5, 11, 11,  8, 10,  6,
          2,  0,  0],
        [11, 12, 12,  5,  7,  8,  4, 12,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10,  4,  7,  5, 11, 10,  5, 12,  7, 12, 11, 12, 11,  2,  0,  0,  0,  0,
          0,  0,  0],
        [12, 10,  5, 10, 12, 11,  9,  5, 10,  4, 12,  4,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  4, 10,  9,  5,  8,  4,  9,  7,  6,  4,  7,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12, 11,  8, 11,  4, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11,  5, 10,  9,  4,  7,  5, 11,  4, 12,  5,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11,  5,  6,  5, 10, 10,  8,  5,  7,  8, 12,  4,  8,  8,  4,  4,
          4,  4,  4],
        [ 9, 10, 12,  8, 12,  4,  5, 12,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5, 11,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5,  6,  5, 11, 11,  7,  8,  8,  6,  4,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10, 11, 11,  7, 11,  8,  4,  5,  7, 11, 10,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11,  4,  7,  9, 11,  6,  8,  5,  4,  8,  9, 12, 12,  4,  6, 12,  8,  2,
          0,  0,  0],
        [10,  9,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  4, 11,  7, 10, 11,  8,  8,  9,  4,  8,  4, 10,  6,  8,  4,  2,
          0,  0,  0],
        [ 6,  7,  4, 10, 11,  4, 12,  9,  7, 10, 11,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0]])


In [126]:
trg = torch.tensor([[11, 10, 11,  6,  6, 11, 12,  6, 12,  4, 10,  6, 11,  4,  5,  4,  7,  2,
          0,  0,  0],
        [ 6,  6,  7, 11,  5, 12, 10,  6, 11, 11, 11,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  6,  6, 12, 10,  4,  9,  8,  8, 10,  9, 12,  9,  4, 12,  6, 11,  6,
          2,  0,  0],
        [ 8, 10, 12,  4, 12,  9,  4, 11,  9,  6,  4,  6,  7, 12,  2,  0,  0,  0,
          0,  0,  0],
        [ 4,  6,  9,  6, 12,  8, 11,  9,  7, 10,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9, 10, 11,  7,  8,  5,  7,  5,  8,  4, 12, 12,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  8,  4,  8,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5, 11,  4, 12,  8, 12,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6,  5,  5, 11,  4, 10,  4,  6,  8,  7,  6,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  4,  4,  5,  8,  6, 12,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  6,  6,  7,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  8,  9,  7,  5,  5,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 8,  4,  9, 10,  7,  4, 11,  8, 11, 10, 12,  4,  5,  8,  5,  7, 10,  2,
          0,  0,  0],
        [ 7, 10,  8,  9,  5,  9,  4, 10, 11, 10,  8,  6,  5, 11, 11,  8, 10,  6,
          2,  0,  0],
        [11, 12, 12,  5,  7,  8,  4, 12,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10,  4,  7,  5, 11, 10,  5, 12,  7, 12, 11, 12, 11,  2,  0,  0,  0,  0,
          0,  0,  0],
        [12, 10,  5, 10, 12, 11,  9,  5, 10,  4, 12,  4,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  4, 10,  9,  5,  8,  4,  9,  7,  6,  4,  7,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12, 11,  8, 11,  4, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11,  5, 10,  9,  4,  7,  5, 11,  4, 12,  5,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11,  7,  6,  5, 10,  8,  5,  7,  8, 12,  4,  9,  8,  8,  4,  9,
          4,  4,  2],
        [ 9, 10, 12,  8, 12,  4,  5, 12,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5, 11,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5,  6,  5, 11, 11,  7,  8,  8,  6,  4,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10, 11, 11,  7, 11,  8,  4,  5,  7, 11, 10,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11,  4,  7,  9, 11,  6,  8,  5,  4,  8,  9, 12, 12,  4,  6, 12,  8,  9,
          2,  0,  0],
        [10,  9,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  4, 11,  7, 10, 11,  8,  8,  9,  4,  8,  4, 10,  6,  8,  4,  2,
          0,  0,  0],
        [ 6,  7,  4, 10, 11,  4, 12,  9,  7, 10, 11,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0]])

In [132]:
correct = 0
correct += list(pred.ne(trg.view_as(pred)).sum(1).numpy()).count(0)

In [133]:
correct

29

In [135]:
list(pred.ne(trg.view_as(pred)).sum(1).numpy()).count(0)

29

In [130]:
pred.size()

torch.Size([32, 21])

In [131]:
pred.ne(trg)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [93]:
out = torch.tensor([[ 7,  3, 10, 12, 10, 10, 12,  1, 12, 10,  3,  5,  1,  3,  3,  3,  3,  3,
          1,  3,  3],
        [ 3,  2,  5, 12,  5, 12,  2,  5,  3,  3,  3,  3,  3,  2,  3,  2,  3,  3,
          2,  5,  3],
        [ 4, 10, 12, 10,  3,  2, 10,  2,  2,  2,  3,  3,  3,  2,  3,  3,  3,  3,
          2,  5,  5],
        [ 4,  1, 10,  8, 12, 12,  3, 12, 12,  5,  5,  5, 12,  2,  5,  3,  3,  3,
          9,  5,  3],
        [ 7,  7, 10,  0, 10,  7, 12,  3,  7, 10, 10,  0,  3,  7, 10, 10,  7, 10,
          8, 10,  7],
        [ 7, 10, 10, 12,  7,  3,  3,  7, 12,  7,  7, 10,  3, 12, 10,  1, 10,  7,
          3,  3, 10],
        [ 4, 12,  5, 12,  1, 12,  1, 12,  2, 10,  2,  3,  3,  3,  3,  3,  3,  3,
          1,  1,  5],
        [ 5,  2,  2,  3,  1,  3,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          1,  5,  3],
        [12,  2, 12,  2,  5,  5,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  2,  3,
          9,  3,  3],
        [ 5,  2,  5,  2,  2,  3,  3,  3,  3,  5,  3,  3,  5,  3,  2,  3,  5,  5,
          2,  5,  5],
        [ 3, 10,  3,  2,  2,  1, 12,  5, 12,  2, 10,  1,  8, 12,  3,  2,  2,  3,
          8,  3,  3],
        [ 4,  5,  2,  5, 12,  2,  5,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          2,  5,  5],
        [ 7,  1, 10, 12, 10, 12,  1, 10, 10, 12, 10, 10, 10, 10,  3,  5,  3,  1,
          7,  3,  3],
        [ 4, 12, 10, 10,  3, 10, 12, 12, 10, 12,  1,  1, 12,  1,  7, 12,  7, 10,
         10, 12,  3],
        [ 4,  3,  2,  5, 12,  2,  2,  2, 12, 12,  3,  2,  3,  3,  2,  3,  3,  3,
          1,  3,  3],
        [ 4, 10, 12,  0, 12,  2,  2,  3,  7, 12, 10,  1,  3,  3,  3,  3,  3,  3,
          9,  3,  3],
        [ 7, 12,  5, 12, 12,  5,  1,  5,  1, 12,  2,  5,  3,  3,  3,  3,  3,  3,
          9,  3,  5],
        [ 4, 12, 10,  1,  5,  5,  5, 12,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
          1,  3,  3],
        [ 5,  2,  2,  5, 12, 12, 12,  1, 12,  5,  3,  3,  3,  3,  3,  3,  3,  3,
          2,  5,  3],
        [ 4,  5,  2,  1,  2,  5,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          2,  3,  3],
        [ 5,  2, 12,  2,  5,  3,  5,  5,  5,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          1,  5,  3],
        [ 4,  1, 12,  1,  1, 10,  3, 12, 12, 12, 12, 12,  0,  3, 12,  5,  3,  1,
          3,  2,  3],
        [ 8,  3,  6,  8,  1, 10,  1,  3,  1, 12, 12, 12,  7, 12, 10, 12,  6,  1,
          8,  3,  2],
        [ 7,  3, 10,  7,  3, 10,  3, 10, 10,  7, 10,  3, 12, 12, 12, 10, 10, 10,
         10,  7,  3],
        [ 4,  3,  1, 12, 12,  1,  3,  5,  1,  5, 10,  5, 10,  2,  3,  3,  3,  3,
          2,  3,  3],
        [ 3, 12,  3,  3,  3,  2,  3,  2,  2,  2,  3,  2,  3,  3,  3,  3,  3,  3,
          7,  3,  3],
        [ 7, 12,  3, 10,  8, 12,  3, 12, 12,  7, 10, 10, 10, 12,  7, 10,  5, 12,
         12,  3,  3],
        [ 5, 12, 12, 12,  5,  5,  5, 12, 12,  3,  3,  3,  3,  5,  3,  3,  3,  3,
          5,  3,  3],
        [ 4,  5, 12,  2,  2,  5,  5,  2,  2,  3,  3,  3,  3,  3,  3,  5,  3,  3,
          2,  5,  5],
        [ 5, 10,  5,  2,  3,  5,  5,  3,  3,  3,  5,  5,  3,  5,  3,  3,  3,  3,
          5,  5,  5],
        [ 4,  1, 10,  5, 12,  3,  2,  3,  8,  3,  8, 10,  5,  2,  3,  3,  3,  3,
          2,  5,  3],
        [ 4,  3, 12,  7,  3, 10,  1, 10, 10, 10, 12,  1,  1, 12,  1, 12,  5, 12,
         12,  3,  3]])

In [94]:
out.size()

torch.Size([32, 21])

In [98]:
lst = [0 if d.item()==2 else d.item() for i in out for d in i]
len(lst)

672

In [104]:
np.array(lst).reshape(32,21).tolist()

[[7, 3, 10, 12, 10, 10, 12, 1, 12, 10, 3, 5, 1, 3, 3, 3, 3, 3, 1, 3, 3],
 [3, 0, 5, 12, 5, 12, 0, 5, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 0, 5, 3],
 [4, 10, 12, 10, 3, 0, 10, 0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, 0, 5, 5],
 [4, 1, 10, 8, 12, 12, 3, 12, 12, 5, 5, 5, 12, 0, 5, 3, 3, 3, 9, 5, 3],
 [7, 7, 10, 0, 10, 7, 12, 3, 7, 10, 10, 0, 3, 7, 10, 10, 7, 10, 8, 10, 7],
 [7, 10, 10, 12, 7, 3, 3, 7, 12, 7, 7, 10, 3, 12, 10, 1, 10, 7, 3, 3, 10],
 [4, 12, 5, 12, 1, 12, 1, 12, 0, 10, 0, 3, 3, 3, 3, 3, 3, 3, 1, 1, 5],
 [5, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 5, 3],
 [12, 0, 12, 0, 5, 5, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 9, 3, 3],
 [5, 0, 5, 0, 0, 3, 3, 3, 3, 5, 3, 3, 5, 3, 0, 3, 5, 5, 0, 5, 5],
 [3, 10, 3, 0, 0, 1, 12, 5, 12, 0, 10, 1, 8, 12, 3, 0, 0, 3, 8, 3, 3],
 [4, 5, 0, 5, 12, 0, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 5, 5],
 [7, 1, 10, 12, 10, 12, 1, 10, 10, 12, 10, 10, 10, 10, 3, 5, 3, 1, 7, 3, 3],
 [4, 12, 10, 10, 3, 10, 12, 12, 10, 12, 1, 1, 12, 1, 7, 12, 7, 10, 10, 12, 3],
 [

In [111]:
[b.index(3) for b in np.array(lst).reshape(32,21).tolist()]

[1,
 0,
 4,
 6,
 7,
 5,
 11,
 3,
 9,
 5,
 0,
 7,
 14,
 4,
 1,
 7,
 12,
 10,
 10,
 7,
 5,
 6,
 1,
 1,
 1,
 0,
 2,
 9,
 9,
 4,
 5,
 1]

In [108]:
[1,2,3,4,2,2,2,].index(2)

1

In [87]:
pred = torch.tensor([[11, 10, 11,  6,  6, 11, 12,  6, 12,  4, 10,  6, 11,  4,  5,  4,  7,  2,
          2,  2,  2],
        [ 6,  6,  7, 11,  5, 12, 10,  6, 11, 11, 11,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 7,  6,  6, 12, 10, 10,  9,  8,  8, 10,  9, 12,  9,  4,  4, 12,  6, 11,
          6,  2,  2],
        [ 8, 10, 12,  4, 12,  9,  4, 11,  9,  9,  6,  4,  7, 12,  2,  2,  2,  2,
          2,  2,  2],
        [ 4,  6,  9,  6, 12,  8, 11,  9,  7, 10,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 9, 10, 11,  7,  8,  5,  7,  5,  8,  4, 12, 12,  5,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 7,  8,  4,  8,  6,  2,  2,  7,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 5, 11,  4, 12,  8, 12,  9, 10,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 5,  6,  5,  5, 11,  4, 10,  4,  6,  8,  7,  6,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 5,  6, 11, 12,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 5,  4,  4,  5,  8,  6, 12,  5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 9,  6,  6,  7,  5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [11, 11, 12,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 4,  8,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [12,  7,  8,  9,  7,  5,  5,  6,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 8,  4,  9, 10,  7,  4, 11,  8, 11, 10, 10, 12,  5,  5,  8,  7, 10,  2,
          2,  2,  2],
        [ 7, 10,  8,  9,  5,  9,  4, 10, 11, 10,  8,  6,  5, 11, 11, 11,  8, 10,
          6,  2,  2],
        [11, 12, 12,  5,  7,  8,  4, 12,  6,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [10,  4,  7,  5, 11, 10,  5, 12,  7, 12, 11, 12, 11,  2,  2,  2,  2,  2,
          2,  2,  2],
        [12, 10,  5, 10, 12, 11,  9,  5, 10, 10, 12,  4,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 4,  4, 10,  9,  5,  8,  4,  9,  7,  6,  4,  7,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [12, 11,  8, 11,  4, 11, 12,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [11, 11,  5, 10,  9,  4,  7,  5, 11,  4, 12,  5,  5,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 5,  6, 11,  7,  6,  5, 10, 10,  5,  7,  7,  8, 12,  8,  9,  8,  8,  4,
          9,  4,  4],
        [ 9, 10, 12,  8, 12,  4,  5, 12,  8,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 9,  5, 11,  8,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [ 9,  5,  6,  5, 11, 11,  7,  8,  8,  6,  4,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [10, 11, 11,  7, 11,  8,  4,  5,  7, 11, 10,  9,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [11,  4,  7,  9, 11,  6,  8,  5,  4,  8,  9, 12, 12, 12,  6, 12,  8,  9,
          2,  2,  2],
        [10,  9,  5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [12,  7,  4, 11,  7, 10, 11,  8,  8,  9,  4,  8,  4, 10, 10,  6,  8,  4,
          2,  2,  2],
        [ 6,  7,  4, 10, 11,  4, 12,  9,  7, 10, 11,  9,  2,  2,  2,  2,  2,  2,
          2,  2,  2]])

In [92]:
[0 if d.item()==2 else d.item() for i in pred for d in i]

[11,
 10,
 11,
 6,
 6,
 11,
 12,
 6,
 12,
 4,
 10,
 6,
 11,
 4,
 5,
 4,
 7,
 0,
 0,
 0,
 0,
 6,
 6,
 7,
 11,
 5,
 12,
 10,
 6,
 11,
 11,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 6,
 6,
 12,
 10,
 10,
 9,
 8,
 8,
 10,
 9,
 12,
 9,
 4,
 4,
 12,
 6,
 11,
 6,
 0,
 0,
 8,
 10,
 12,
 4,
 12,
 9,
 4,
 11,
 9,
 9,
 6,
 4,
 7,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 6,
 9,
 6,
 12,
 8,
 11,
 9,
 7,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 10,
 11,
 7,
 8,
 5,
 7,
 5,
 8,
 4,
 12,
 12,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 4,
 8,
 6,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 11,
 4,
 12,
 8,
 12,
 9,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 5,
 5,
 11,
 4,
 10,
 4,
 6,
 8,
 7,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 6,
 11,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 4,
 4,
 5,
 8,
 6,
 12,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 6,
 6,
 7,
 5,
 0,
 0,
 0,
 0

In [86]:
for i in pred.numpy():
    print(list(i))

[11, 10, 11, 6, 6, 11, 12, 6, 12, 4, 10, 6, 11, 4, 5, 4, 7, 2, 2, 2, 2]
[6, 6, 7, 11, 5, 12, 10, 6, 11, 11, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[7, 6, 6, 12, 10, 10, 9, 8, 8, 10, 9, 12, 9, 4, 4, 12, 6, 11, 6, 2, 2]
[8, 10, 12, 4, 12, 9, 4, 11, 9, 9, 6, 4, 7, 12, 2, 2, 2, 2, 2, 2, 2]
[4, 6, 9, 6, 12, 8, 11, 9, 7, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[9, 10, 11, 7, 8, 5, 7, 5, 8, 4, 12, 12, 5, 2, 2, 2, 2, 2, 2, 2, 2]
[7, 8, 4, 8, 6, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[5, 11, 4, 12, 8, 12, 9, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[5, 6, 5, 5, 11, 4, 10, 4, 6, 8, 7, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[5, 6, 11, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[5, 4, 4, 5, 8, 6, 12, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[9, 6, 6, 7, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[11, 11, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[4, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[12, 7, 8, 9, 7, 5, 5, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [71]:
trg = torch.tensor([[11, 10, 11,  6,  6, 11, 12,  6, 12,  4, 10,  6, 11,  4,  5,  4,  7,  2,
          0,  0,  0],
        [ 6,  6,  7, 11,  5, 12, 10,  6, 11, 11, 11,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  6,  6, 12, 10,  4,  9,  8,  8, 10,  9, 12,  9,  4, 12,  6, 11,  6,
          2,  0,  0],
        [ 8, 10, 12,  4, 12,  9,  4, 11,  9,  6,  4,  6,  7, 12,  2,  0,  0,  0,
          0,  0,  0],
        [ 4,  6,  9,  6, 12,  8, 11,  9,  7, 10,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9, 10, 11,  7,  8,  5,  7,  5,  8,  4, 12, 12,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  8,  4,  8,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5, 11,  4, 12,  8, 12,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6,  5,  5, 11,  4, 10,  4,  6,  8,  7,  6,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  4,  4,  5,  8,  6, 12,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  6,  6,  7,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  8,  9,  7,  5,  5,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 8,  4,  9, 10,  7,  4, 11,  8, 11, 10, 12,  4,  5,  8,  5,  7, 10,  2,
          0,  0,  0],
        [ 7, 10,  8,  9,  5,  9,  4, 10, 11, 10,  8,  6,  5, 11, 11,  8, 10,  6,
          2,  0,  0],
        [11, 12, 12,  5,  7,  8,  4, 12,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10,  4,  7,  5, 11, 10,  5, 12,  7, 12, 11, 12, 11,  2,  0,  0,  0,  0,
          0,  0,  0],
        [12, 10,  5, 10, 12, 11,  9,  5, 10,  4, 12,  4,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 4,  4, 10,  9,  5,  8,  4,  9,  7,  6,  4,  7,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12, 11,  8, 11,  4, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11, 11,  5, 10,  9,  4,  7,  5, 11,  4, 12,  5,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11,  7,  6,  5, 10,  8,  5,  7,  8, 12,  4,  9,  8,  8,  4,  9,
          4,  4,  2],
        [ 9, 10, 12,  8, 12,  4,  5, 12,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5, 11,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9,  5,  6,  5, 11, 11,  7,  8,  8,  6,  4,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [10, 11, 11,  7, 11,  8,  4,  5,  7, 11, 10,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [11,  4,  7,  9, 11,  6,  8,  5,  4,  8,  9, 12, 12,  4,  6, 12,  8,  9,
          2,  0,  0],
        [10,  9,  5,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [12,  7,  4, 11,  7, 10, 11,  8,  8,  9,  4,  8,  4, 10,  6,  8,  4,  2,
          0,  0,  0],
        [ 6,  7,  4, 10, 11,  4, 12,  9,  7, 10, 11,  9,  2,  0,  0,  0,  0,  0,
          0,  0,  0]])

In [72]:
trg

tensor([[11, 10, 11,  6,  6, 11, 12,  6, 12,  4, 10,  6, 11,  4,  5,  4,  7,  2,
          0,  0,  0],
        [ 6,  6,  7, 11,  5, 12, 10,  6, 11, 11, 11,  2,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  6,  6, 12, 10,  4,  9,  8,  8, 10,  9, 12,  9,  4, 12,  6, 11,  6,
          2,  0,  0],
        [ 8, 10, 12,  4, 12,  9,  4, 11,  9,  6,  4,  6,  7, 12,  2,  0,  0,  0,
          0,  0,  0],
        [ 4,  6,  9,  6, 12,  8, 11,  9,  7, 10,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 9, 10, 11,  7,  8,  5,  7,  5,  8,  4, 12, 12,  5,  2,  0,  0,  0,  0,
          0,  0,  0],
        [ 7,  8,  4,  8,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5, 11,  4, 12,  8, 12,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6,  5,  5, 11,  4, 10,  4,  6,  8,  7,  6,  2,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 5,  6, 11, 12,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [73]:
if 2 in [1,2,3]:
    print('yes')

yes


In [112]:
x = torch.Tensor(2,3)
y = x.permute(1,0)
y.view(-1) # 报错，因为x和y指针指向相同

RuntimeError: invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view(). at /Users/administrator/nightlies/pytorch-1.0.0/wheel_build_dirs/wheel_3.6/pytorch/aten/src/TH/generic/THTensor.cpp:213

In [ ]:
y = x.permute(1,0).contiguous()
y.view(-1) # OK

In [116]:
teacher_force = random.random() < 0.85
teacher_force

True

In [124]:
random.random()

0.43342949588980095

In [80]:
lst = [1,2,3,0,0,0]

In [81]:
idx = lst.index(0)
idx

3

In [82]:
lst[idx] =2

In [83]:
lst

[1, 2, 3, 2, 0, 0]

In [37]:
r=torch.tensor([[ 1.1157,  1.3396,  0.6192,  0.3732,  0.8985],[ 1.1157,  1.3396,  0.6192,  0.3732,  0.8985],[ 1.1157,  1.3396,  0.6192,  0.3732,  0.8985]])

In [38]:
r

tensor([[1.1157, 1.3396, 0.6192, 0.3732, 0.8985],
        [1.1157, 1.3396, 0.6192, 0.3732, 0.8985],
        [1.1157, 1.3396, 0.6192, 0.3732, 0.8985]])

In [41]:
r.size()

torch.Size([3, 5])

In [45]:
s=torch.tensor([[ 1,  1.3396,  0.6192,  0.3732,  0.8985],[ 1.1157,  1.3396,  0.6192,  0.3732,  0.8985],[ 1.1157,  1.3396,  0.6192,  0.3732,  0.8985]])

In [46]:
r.equal(s)

False

In [28]:
x = torch.randn(32, 21)
x

tensor([[ 5.7535e-01,  1.3838e-01,  1.2405e+00,  1.4840e+00,  1.0962e+00,
          8.1263e-01, -1.5058e+00,  2.3466e-01, -5.9770e-01,  1.1814e+00,
         -6.4241e-01,  1.5214e+00,  2.3960e+00,  2.4335e+00, -1.5810e-01,
         -1.0320e+00, -4.3553e-01,  1.3897e-01, -9.3737e-01, -7.1900e-01,
          4.6919e-01],
        [-7.0006e-02,  9.7391e-01, -6.8093e-01,  7.2759e-02, -1.9988e-01,
         -4.3166e-01,  3.8335e-01,  6.7644e-01, -3.2726e-01,  1.2362e+00,
          9.1858e-01, -7.2289e-01,  2.8124e-01, -8.8433e-01,  1.7142e+00,
          1.5038e+00, -5.1785e-01,  7.8735e-02,  9.1024e-01, -2.4034e-01,
         -1.3185e+00],
        [-9.4349e-02,  7.9184e-01, -1.0125e+00,  4.0152e-02,  1.6738e+00,
         -4.1083e-01,  1.5856e+00, -1.0693e-01, -9.2871e-02, -1.2224e+00,
          1.9448e-01,  1.5264e+00,  1.1978e+00,  1.7077e+00,  1.4378e+00,
          7.6805e-01, -1.9883e-01, -1.4481e+00, -5.0010e-01, -9.9813e-01,
         -9.6106e-01],
        [-1.4803e+00, -2.3627e+00, -5.8017e

In [57]:
list(torch.ne(torch.Tensor([[1, 2,3], [3, 4,5]]), torch.Tensor([[1,1,1], [3, 4,5]])).sum(1).numpy()).count(0)

1

In [68]:
torch.Tensor([[1, 2,3], [3, 4,5]]).size()

torch.Size([2, 3])

In [67]:
torch.Tensor([[1, 2,3], [3, 4,5]]).ne(torch.Tensor([[1,1,1], [3, 4,5]])).sum(1)

tensor([2, 0])

In [61]:
list(torch.Tensor([[1, 2,3], [3, 4,5]]).ne(torch.Tensor([[1,1,1], [3, 4,5]])).sum(1).numpy()).count(0)

1

In [22]:
y = torch.randn(32, 21)

In [27]:
x.eq(y).sum(1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [65]:
from models import Encoder, Decoder, Attention, Seq2Seq

ModuleNotFoundError: No module named 'models'

In [66]:
from dataset import Digits, DigitSeqDataset, DigitSeqTestDataset

ImportError: cannot import name 'Digits'

In [4]:
from utils import build_vocab, load_datasets

### 加载数据集合

In [62]:
train_input_csv = '../dataset/task8_train_input.csv'
train_output_csv = '../dataset/task8_train_output.csv'
test_input_csv = '../dataset/task8_test_input.csv'

In [63]:
train_inputs, train_outputs, test_inputs = load_datasets(train_input_csv, train_output_csv, test_input_csv)

NameError: name 'load_datasets' is not defined

### 训练集与验证集划分

In [7]:
# Split train and valid datasets. ratio = 0.3
train_inputs, valid_inputs, train_outputs, valid_outputs = train_test_split(train_inputs, train_outputs, test_size=0.3, random_state=42)

In [8]:
len(train_inputs),len(valid_inputs)

(22400, 9600)

In [9]:
digit2index, index2digit, vocab_size = build_vocab('123456789')

In [10]:
digit2index

{'1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '<EOS>': 2,
 '<GO>': 1,
 '<PAD>': 0,
 '<UNK>': 3}

In [11]:
train_input_output_dataset = DigitSeqDataset(train_inputs, train_outputs)

In [12]:
valid_input_output_dataset = DigitSeqDataset(valid_inputs, valid_outputs)

In [13]:
test_input_dataset = DigitSeqDataset(test_inputs, valid_outputs)

In [14]:
len(test_input_dataset)

32000

In [15]:
train_loader = torch.utils.data.DataLoader(train_input_output_dataset, batch_size=32, shuffle=False)

In [16]:
valid_loader = torch.utils.data.DataLoader(valid_input_output_dataset, batch_size=32, shuffle=False)

### 构建模型

### Training the Seq2Seq Model

In [17]:
INPUT_DIM = len(digit2index)
OUTPUT_DIM = len(digit2index)

ENC_EMB_DIM = 8
DEC_EMB_DIM = 8

ENC_HID_DIM = 32
DEC_HID_DIM = 32

ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [18]:
optimizer = optim.Adam(model.parameters())

In [19]:
pad_idx = 0

criterion = nn.CrossEntropyLoss(ignore_index=0)

In [25]:
def train(model, iterator, digit2index,optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.get('input_seq_tensor').squeeze().permute(1,0)#.view(22, 32)#.view(32,-1,1)
        trg = batch.get('output_seq_tensor').squeeze().permute(1,0)#.view(22, 32)#.view(32,-1,1)
    
        optimizer.zero_grad()
        
        output = model(src, trg, digit2index)

        loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].contiguous().view(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def evaluate(model, iterator, digit2index, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            
            src = batch.get('input_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)
            trg = batch.get('output_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)

            #turn off teacher forcing
            output = model(src, trg, 0, digit2index) 

            loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].contiguous().view(-1))

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [29]:
def predict(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    preds = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            
            src = batch.get('input_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)
            # trg = batch.get('output_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)

            trg = None

            #turn off teacher forcing
            output = model(src, trg, 0) 
            
            topv, topi = output.data.topk(1)

            decoder_output = topi.squeeze().detach()
            decoder_output = decoder_output[1:].permute(1, 0)

            preds.append([digit_idx.item() for seq in decoder_output for digit_idx in seq])
        
    return preds

In [30]:
index2digit[0]

'<PAD>'

In [33]:
digit2index

{'1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 '<EOS>': 2,
 '<GO>': 1,
 '<PAD>': 0,
 '<UNK>': 3}

In [32]:
EPOCHS = 1

CLIP = 10

SAVE_DIR = './checkpoints/'

MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'seq2seq_attention_model.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
    
train_iterator = train_loader
valid_iterator = valid_loader


for epoch in range(EPOCHS):
    
    train_loss = train(model, train_iterator, digit2index, optimizer, criterion, CLIP)
    
    valid_loss = evaluate(model, valid_iterator, digit2index, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f} |')

AttributeError: 'int' object has no attribute 'get'

In [ ]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [ ]:
model

In [ ]:
test_input_csv = './dataset/task8_test_input.csv'
test_inputs = pd.read_csv(test_input_csv, header=None)

In [ ]:
test_input_dataset = DigitSeqTestDataset(test_inputs)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_input_dataset, batch_size=32, shuffle=False)

In [ ]:
len(test_loader)

In [ ]:
test_iterator = test_loader

In [ ]:
test_result = predict(model, test_iterator,criterion)

In [ ]:
len(test_result)

In [ ]:
pred_array = np.array([index2digit[item] for batch in test_result for item in batch]).reshape(1000,32,21).reshape(32000,21)

In [ ]:
pd.DataFrame([list("".join(list(i)).replace('<EOS>','0')[:-1]) for i in pred_array]).to_csv('reverse_seq_predictions .csv', header=None,index=False)

In [161]:
def pad_sentence(digits_sequence, max_digits_len=20, pad_token_id=0):
        '''
        对 batch 中的 digits序列进行补全，保证 batch 中的每行都有相同的 sequence_length
        参数：
        - sentence batch
        - pad_int: <PAD>对应索引号
        '''
        # max_digits = max([len(digits) for digits in digits_seq_batch])
        return digits_sequence + [pad_token_id] * (max_digits_len - len(digits_sequence))

In [ ]:
input_digit2index_seq = self.pad_sentence(input_digit2index_seq, 22, digit2index.get('<PAD>'))

In [147]:
df_test_input.shape[0]

32000

In [137]:
df_test_input = pd.read_csv('./dataset/task8_test_input.csv', header=None)

In [151]:
list(reversed(df_test_input.iloc[0, :]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 8, 9, 8, 9, 4, 2, 7, 6, 3, 7]

In [153]:
list(df_test_input.iloc[0, :])

[7, 3, 6, 7, 2, 4, 9, 8, 9, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [155]:
[digit for digit in df_test_input.iloc[0, :] if digit != 0]

[7, 3, 6, 7, 2, 4, 9, 8, 9, 8, 1]

In [181]:
test_output = []

In [182]:
for index in range(df_test_input.shape[0]):
    digits_seq = list(reversed([digit for digit in df_test_input.iloc[index, :] if digit != 0]))
#     print(digits_seq)
    digits_seq = pad_sentence(digits_seq, 20, 0)
    test_output.append(digits_seq)

In [183]:
len(test_output)

32000

In [184]:
pd.DataFrame(test_output).to_csv('task8_test_output.csv', header=None,index=False)

In [ ]:
df_test_input = pd.read_csv('./dataset/task8_test_input.csv', header=None)

In [227]:
df_test_output = pd.read_csv('./task8_test_output.csv', header=None)
df_test_output_pred = pd.read_csv('./pytorch_seq2seq_attention_v2/reverse_seq_predictions .csv', header=None)

In [223]:
df_test_input.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,7,3,6,7,2,4,9,8,9,8,1,0,0,0,0,0,0,0,0,0
1,9,6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,8,3,2,9,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,5,2,7,7,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,7,2,6,3,1,1,3,4,6,2,4,8,4,4,3,9,0,0,0


In [215]:
df_test_output.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,8,9,8,9,4,2,7,6,3,7,0,0,0,0,0,0,0,0,0
1,2,1,6,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,4,9,2,3,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,1,7,7,2,5,9,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,3,4,4,8,4,2,6,4,3,1,1,3,6,2,7,3,0,0,0


In [216]:
df_test_output_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,8,9,8,9,4,2,7,6,3,7,0,0,0,0,0,0,0,0,0
1,2,1,6,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,4,9,2,3,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8,1,7,7,2,5,9,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,3,4,4,8,4,2,6,4,3,1,1,3,6,2,7,3,0,0,0


In [228]:
true_seq = df_test_output.as_matrix()

In [229]:
pred_seq = df_test_output_pred.as_matrix()

In [226]:
list((true_seq == pred_seq).all(1)).count(True) / pred_seq.shape[0]

0.95384375

In [230]:
list((true_seq == pred_seq).all(1)).count(True) / pred_seq.shape[0]

0.9963125

In [221]:
pred_seq.shape

(32000, 20)